In [1]:
# =========================================================
# 🔧 Configuration initiale : Seed, Device, Chemins
# =========================================================
import numpy as np, random, os
import torch

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Seed={SEED} | Device={DEVICE}")
torch.cuda.empty_cache() if DEVICE=="cuda" else None


✅ Seed=42 | Device=cuda


In [2]:
# =========================================================
# 📁 Configuration des chemins (standardisée)
# =========================================================
import os
from pathlib import Path
import sys

# --- Détection du projet (exécution depuis notebooks/) ---
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
DATAGENERATOR_PATH = PROJECT_ROOT / "p9dg"

# Dossiers globaux du projet
MODELS_DIR      = PROJECT_ROOT / "models"
ARTIFACTS_DIR   = PROJECT_ROOT / "artifacts"
OUTPUTS_DIR     = PROJECT_ROOT / "outputs"
DATA_ROOT       = Path(os.getenv("DATA_ROOT", PROJECT_ROOT / "data")).resolve()
CONFIG_DIR      = Path(os.getenv("CONFIG_DIR", PROJECT_ROOT / "configs")).resolve()

# Dossiers spécifiques pour les métriques
METRICS_DIR     = ARTIFACTS_DIR / "metrics"
METRICS_DIR.mkdir(parents=True, exist_ok=True)

# Création des dossiers
for d in [MODELS_DIR, ARTIFACTS_DIR, OUTPUTS_DIR, CONFIG_DIR, METRICS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Ajout du datagenerator au path
if str(DATAGENERATOR_PATH) not in sys.path:
    sys.path.append(str(DATAGENERATOR_PATH))

# Ajout de la racine du projet au path pour les imports
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Imports package-qualifiés
from p9dg.histo_dataset import HistoDataset
from metrics.fid_lpips_eval import FIDLPIPSEvaluator, run_eval_experiment, run_eval_paired_experiment

# Affichage des dossiers configurés
print("✅ Dossiers configurés:")
print(f"   PROJECT_ROOT    : {PROJECT_ROOT}")
print(f"   DATA_ROOT       : {DATA_ROOT}")
print(f"   CONFIG_DIR      : {CONFIG_DIR}")
print(f"   MODELS_DIR      : {MODELS_DIR}")
print(f"   ARTIFACTS_DIR   : {ARTIFACTS_DIR}")
print(f"   OUTPUTS_DIR     : {OUTPUTS_DIR}")
print(f"   METRICS_DIR     : {METRICS_DIR}")
print("✅ Imports réussis: HistoDataset, FIDLPIPSEvaluator")

✅ Dossiers configurés:
   PROJECT_ROOT    : /workspace
   DATA_ROOT       : /workspace/data
   CONFIG_DIR      : /workspace/configs
   MODELS_DIR      : /workspace/models
   ARTIFACTS_DIR   : /workspace/artifacts
   OUTPUTS_DIR     : /workspace/outputs
   METRICS_DIR     : /workspace/artifacts/metrics
✅ Imports réussis: HistoDataset, FIDLPIPSEvaluator


In [3]:
# ==========================================================
# 🧮 Classe d’évaluation FID + LPIPS multi-classes
# ==========================================================

import pandas as pd
import numpy as np
from tqdm import tqdm
from p9dg.utils.class_mappings import class_labels
import random, torch, warnings
from torch_fidelity import calculate_metrics
import lpips
from torchvision import transforms
from PIL import Image
import tempfile, shutil, os

import warnings

# --- Masquer les warnings non critiques ---
warnings.filterwarnings("ignore", category=FutureWarning, module="lpips.lpips")
warnings.filterwarnings("ignore", category=UserWarning, module="torch_fidelity.datasets")

In [4]:
# --- UNI2-h embedder (timm + HF Hub, même archi que 07/08) ---
import os, numpy as np, torch, timm
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

def make_uni2h_embedder_timm(device=None, hf_token=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    # Autorisation Hugging Face pour timm (hf-hub:...)
    hf_token = hf_token or os.environ.get("HF_TOKEN") or os.environ.get("HUGGINGFACE_HUB_TOKEN")
    if hf_token and "HUGGINGFACE_HUB_TOKEN" not in os.environ:
        os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token

    timm_kwargs = {
        "img_size": 224,
        "patch_size": 14,
        "depth": 24,
        "num_heads": 24,
        "embed_dim": 1536,
        "mlp_ratio": 2.66667 * 2,  # UNI-2h
        "init_values": 1e-5,
        "num_classes": 0,
        "no_embed_class": True,
        "reg_tokens": 8,
        "dynamic_img_size": True,
        "mlp_layer": timm.layers.SwiGLUPacked,
        "act_layer": torch.nn.SiLU,
    }

    uni = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs).eval().to(device)
    cfg = resolve_data_config(uni.pretrained_cfg, model=uni)  # idem à 07/08
    tfm = create_transform(**cfg)

    @torch.no_grad()
    def uni_embed(path: str) -> np.ndarray:
        img = Image.open(path).convert("RGB")
        x = tfm(img).unsqueeze(0).to(device)
        feat = uni(x)                # (1, 1536)
        if feat.ndim == 4:           # just-in-case
            feat = feat.mean(dim=(-2, -1))
        return feat.squeeze(0).float().cpu().numpy().astype(np.float64)

    # petit sanity check (désactive si tu veux)
    # print("UNI-2h ready → dim", int(uni.num_features), "| device:", device)
    return uni_embed


In [5]:
from huggingface_hub import login
import os

HF_TOKEN = os.getenv("HF_TOKEN")
# Vérifie qu'il est chargé (ne jamais l'afficher publiquement)
assert HF_TOKEN is not None, "HF_TOKEN n'est pas défini dans l'environnement !"

In [6]:
uni_embed = make_uni2h_embedder_timm() 

In [7]:
# Test 1: Mode dossiers avec FID_UNI
df = run_eval_experiment(
    name="real_vs_pixcell_kid_prc_uni",
    real_root=OUTPUTS_DIR / "pixcell_synth" / "train" / "real",
    gen_root=OUTPUTS_DIR / "pixcell_synth" / "train" / "synth",
    save_dir=METRICS_DIR,
    seed=SEED,
    uni_embed=uni_embed,
    fid_uni_max=50,   # ← sample max pour FID_UNI
    max_images_per_class=50,  # ← idem pour FID/KID/PRC
    lpips_pairs=50,   # ← LPIPS pairé borné
)


🎯 Expérience : real_vs_pixcell_kid_prc_uni
📁 real_root = /workspace/outputs/pixcell_synth/train/real
📁 gen_root  = /workspace/outputs/pixcell_synth/train/synth
📦 classes évaluées : ['ADI', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : real vs synth
📂 Classes évaluées : ADI, DEB, LYM, MUC, MUS, NORM, STR, TUM
💾 Rapport : /workspace/artifacts/metrics/real_synth_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv


Évaluation multi-classes: 100%|██████████| 8/8 [02:12<00:00, 16.54s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/real_synth_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/real_vs_pixcell_kid_prc_uni_metrics.csv


In [8]:
df

,class,n_real,n_synth,FID,KID_mean,KID_std,LPIPS,FID_UNI,experiment,max_images_per_class,lpips_pairs
0,ADI,50,50,324.605324,0.197659,1.884952e-07,0.583401,351.774707,real_vs_pixcell_kid_prc_uni,50,50
1,DEB,50,50,360.086695,0.305777,1.659293e-07,0.686600,320.297837,real_vs_pixcell_kid_prc_uni,50,50
2,LYM,50,50,330.514219,0.316774,1.934120e-07,0.712740,402.193924,real_vs_pixcell_kid_prc_uni,50,50
3,MUC,50,50,331.362883,0.294824,1.734017e-07,0.657867,305.472846,real_vs_pixcell_kid_prc_uni,50,50
4,MUS,50,50,336.253769,0.213358,1.615010e-07,0.660323,410.913382,real_vs_pixcell_kid_prc_uni,50,50
5,NORM,50,50,397.912413,0.315392,1.789366e-07,0.725148,430.095939,real_vs_pixcell_kid_prc_uni,50,50
6,STR,50,50,368.996260,0.292684,1.400833e-07,0.635372,317.360711,real_vs_pixcell_kid_prc_uni,50,50
7,TUM,50,50,304.533535,0.192312,1.695027e-07,0.638846,378.783228,real_vs_pixcell_kid_prc_uni,50,50


In [9]:
# Test 2: Mode pairé avec CSV
import pandas as pd
import inspect

# Chercher le CSV dans différents emplacements possibles
csv_candidates = [
#    OUTPUTS_DIR / "pixcell_synth" / "pairs_real_synth.csv",
    OUTPUTS_DIR / "07_diffusion_model" / "pixcell_out_histo" / "pixcell_metadata_*.csv",
    ARTIFACTS_DIR / "pairs_real_synth.csv",
]

csv_path = None
for candidate in csv_candidates:
    if "*" in str(candidate):
        # Chercher avec glob à partir du parent du pattern
        matches = sorted(candidate.parent.glob(candidate.name))
        if matches:
            csv_path = matches[0]
            break
    elif candidate.exists():
        csv_path = candidate
        break

if csv_path is None:
    print("⚠️ Aucun CSV trouvé, création d'un CSV de test...")
    # Créer un CSV minimal pour les tests
    csv_path = METRICS_DIR / "test_pairs.csv"
    # Le CSV sera créé si nécessaire dans les tests suivants
else:
    print(f"✅ CSV trouvé: {csv_path}")

# 1) Lire et détecter les colonnes
if csv_path.exists():
    dfp = pd.read_csv(csv_path)
    cols = set(dfp.columns.str.lower())

    REAL_CANDS  = ["ref_path","real_path","real","path_real"]
    GEN_CANDS   = ["out_path","gen_path","synth_path","generated","path_gen"]
    CLASS_CANDS = ["ref_label","label","class","cls","category"]

    def pick(cands):
        for c in cands:
            if c in cols: 
                # retourne le nom EXACT présent dans le df (respecte la casse)
                return next(x for x in dfp.columns if x.lower()==c)
        return None

    real_col  = pick(REAL_CANDS)
    gen_col   = pick(GEN_CANDS)
    class_col = pick(CLASS_CANDS) or None  # None -> tout regroupé en 'ALL' (géré par le runner)

    if real_col is None or gen_col is None:
        raise KeyError(f"Colonnes introuvables : real={REAL_CANDS}, gen={GEN_CANDS}\nTrouvées: {list(dfp.columns)}")

    print("Colonnes:", {"real_col":real_col, "gen_col":gen_col, "class_col":class_col})

    # 2) Normaliser les chemins (si relatifs)
    def norm_path(p):
        p = Path(str(p))
        return str(p if p.is_absolute() else (PROJECT_ROOT / p))

    dfp[real_col] = dfp[real_col].map(norm_path)
    dfp[gen_col]  = dfp[gen_col].map(norm_path)

    tmp_csv = METRICS_DIR / "pairs_real_synth.normalized.csv"
    dfp.to_csv(tmp_csv, index=False)

    # 3) Appel robuste (détecte si ta version supporte FID_UNI et lpips_pairs, etc.)
    sig = inspect.signature(run_eval_paired_experiment)
    kwargs = dict(
        name="paired_pixcell_kid_prc_uni",
        pairs_csv=tmp_csv,
        save_dir=METRICS_DIR,
        seed=SEED,
        real_col=real_col,
        gen_col=gen_col,
    )
    if "class_col" in sig.parameters: kwargs["class_col"] = class_col
    if "lpips_pairs" in sig.parameters: kwargs["lpips_pairs"] = 50

    # Active FID_UNI si tu as déjà défini uni_embed (UNI2-h timm). Sinon, il sera ignoré.
    try:
        uni_embed  # noqa
        if "uni_embed" in sig.parameters: kwargs["uni_embed"] = uni_embed
        if "fid_uni_max" in sig.parameters: kwargs["fid_uni_max"] = 50
    except NameError:
        pass

    df_paired = run_eval_paired_experiment(**kwargs)
    df_paired
else:
    print("⚠️ CSV introuvable, test ignoré")


✅ CSV trouvé: /workspace/outputs/07_diffusion_model/pixcell_out_histo/pixcell_metadata_1763635740.csv
Colonnes: {'real_col': 'ref_path', 'gen_col': 'out_path', 'class_col': 'ref_label'}
📑 CSV : /workspace/artifacts/metrics/pairs_real_synth.normalized.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : paired_pixcell_kid_prc_uni  |  Device: cuda
📦 Classes : ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']


Évaluation pairée: 100%|██████████| 9/9 [02:25<00:00, 16.13s/it]


✅ Rapport pairé enregistré : /workspace/artifacts/metrics/paired_pixcell_kid_prc_uni_paired_metrics.csv


In [10]:
df_paired

,class,n_pairs,FID,KID_mean,KID_std,LPIPS,FID_UNI,Precision,Recall
0,ADI,50,139.855971,0.032395,1.305203e-07,0.450707,219.964929,NaN,NaN
1,BACK,50,111.803277,0.023069,1.612554e-07,0.263312,242.112444,NaN,NaN
2,DEB,50,101.905040,0.025436,1.488911e-07,0.404742,196.763217,NaN,NaN
3,LYM,50,81.029595,0.034067,1.758960e-07,0.327944,253.358143,NaN,NaN
4,MUC,50,142.482193,0.013708,1.453584e-07,0.456694,161.691089,NaN,NaN
5,MUS,50,156.852417,0.041560,1.653518e-07,0.411078,257.436283,NaN,NaN
6,NORM,50,161.103947,0.056751,1.869012e-07,0.454551,309.579836,NaN,NaN
7,STR,50,108.689386,0.023189,2.013797e-07,0.410843,206.780352,NaN,NaN
8,TUM,50,134.567216,0.027516,1.908322e-07,0.465834,248.258180,NaN,NaN


In [11]:
# Test 3: FIDLPIPSEvaluator direct (mono-classe)
evaluator = FIDLPIPSEvaluator(
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=OUTPUTS_DIR / "07_diffusion_model" / "pixcell_out_histo",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    seed=SEED
)
df = evaluator.run()


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs pixcell_out_histo
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_pixcell_out_histo_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:15<00:00, 15.60s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_pixcell_out_histo_TUM.csv


In [12]:
display(df)

,class,n_real,n_synth,FID,KID_mean,KID_std,LPIPS,FID_UNI
0,TUM,400,400,75.650615,0.039632,0.007157,0.488061,None


In [13]:
# Test 4: Sanity check - réel vs réel (deux tirages indépendants dans le même dossier)
df = run_eval_experiment(
    name="real_vs_real_TUM",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],  # ou None pour toutes
    drop_back_variant=True,
    max_images_per_class=400,
    lpips_pairs=50,
    seed=SEED
)

display(df)


🎯 Expérience : real_vs_real_TUM
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:12<00:00, 12.83s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/real_vs_real_TUM_metrics.csv


,class,n_real,n_synth,FID,KID_mean,KID_std,LPIPS,FID_UNI,experiment,max_images_per_class,lpips_pairs
0,TUM,400,400,-0.00004,-0.001565,0.00297,0.0,None,real_vs_real_TUM,400,50


In [14]:
# Test 5: Réel vs normalisé 
run_eval_experiment(
    name="real_vs_normalized_tst",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=OUTPUTS_DIR / "preprocessing" / "normalized_tst",
    save_dir=METRICS_DIR,
    classes=None,  # toutes les classes
    max_images_per_class=400,
    drop_back_variant=True,
    lpips_pairs=50,
    seed=SEED
)

🎯 Expérience : real_vs_normalized_tst
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/outputs/preprocessing/normalized_tst
📦 classes évaluées : ['ADI', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs normalized_tst
📂 Classes évaluées : ADI, DEB, LYM, MUC, MUS, NORM, STR, TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_normalized_tst_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv


Évaluation multi-classes: 100%|██████████| 8/8 [01:46<00:00, 13.27s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_normalized_tst_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/real_vs_normalized_tst_metrics.csv


,class,n_real,n_synth,FID,KID_mean,KID_std,LPIPS,FID_UNI,experiment,max_images_per_class,lpips_pairs
0,ADI,400,234,72.266041,0.013956,0.005381,0.515938,None,real_vs_normalized_tst,400,50
1,DEB,400,400,46.051960,0.016499,0.004390,0.438681,None,real_vs_normalized_tst,400,50
2,LYM,400,400,24.937919,0.005874,0.003616,0.340390,None,real_vs_normalized_tst,400,50
3,MUC,400,332,62.448501,0.011907,0.004021,0.516443,None,real_vs_normalized_tst,400,50
4,MUS,400,400,90.221001,0.046691,0.015360,0.541433,None,real_vs_normalized_tst,400,50
5,NORM,400,313,56.352615,0.010777,0.003895,0.476960,None,real_vs_normalized_tst,400,50
6,STR,400,365,54.469280,0.022072,0.006451,0.470280,None,real_vs_normalized_tst,400,50
7,TUM,400,400,52.528122,0.013849,0.004562,0.452923,None,real_vs_normalized_tst,400,50


In [15]:
# Test 6: Réel vs synthétique (UNI-2h+PixCell gated)
run_eval_experiment(
    name="real_vs_UNI_Pixcell_pretrained_unpaired",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=OUTPUTS_DIR / "07_diffusion_model" / "pixcell_out_histo",
    save_dir=METRICS_DIR,
    classes=None,  # toutes les classes
    max_images_per_class=300,
    drop_back_variant=True,
    lpips_pairs=50,
    seed=SEED
)

🎯 Expérience : real_vs_UNI_Pixcell_pretrained_unpaired
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/outputs/07_diffusion_model/pixcell_out_histo
📦 classes évaluées : ['ADI', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs pixcell_out_histo
📂 Classes évaluées : ADI, DEB, LYM, MUC, MUS, NORM, STR, TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_pixcell_out_histo_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv


Évaluation multi-classes: 100%|██████████| 8/8 [01:39<00:00, 12.43s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_pixcell_out_histo_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/real_vs_UNI_Pixcell_pretrained_unpaired_metrics.csv


,class,n_real,n_synth,FID,KID_mean,KID_std,LPIPS,FID_UNI,experiment,max_images_per_class,lpips_pairs
0,ADI,300,300,89.973312,0.035722,0.006167,0.531270,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
1,DEB,300,300,66.907315,0.028825,0.007638,0.531308,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
2,LYM,300,300,57.393346,0.036262,0.005436,0.368316,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
3,MUC,300,300,86.217879,0.026552,0.005423,0.543316,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
4,MUS,300,300,102.296928,0.039298,0.007233,0.567855,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
5,NORM,300,300,102.317033,0.057580,0.009548,0.530825,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
6,STR,300,300,74.653237,0.030879,0.006296,0.513105,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50
7,TUM,300,300,82.215442,0.039617,0.006617,0.494496,None,real_vs_UNI_Pixcell_pretrained_unpaired,300,50


In [16]:
# Test 7: Mode pairé avec CSV PixCell (toutes les paires)
# Chercher le CSV PixCell
pixcell_csvs = list(OUTPUTS_DIR.glob("pixcell_out_histo/pixcell_metadata_*.csv"))
if pixcell_csvs:
    csv_path = sorted(pixcell_csvs)[-1]  # Prendre le plus récent
    print(f"✅ Utilisation du CSV: {csv_path.name}")
    
    df = run_eval_paired_experiment(
        name="paired_UNI_PixCell",
        pairs_csv=csv_path,
        save_dir=METRICS_DIR,
        # class_col="ref_label",      # par défaut
        lpips_pairs=None,             # None = toutes les paires
        seed=SEED
    )
    display(df)
else:
    print("⚠️ Aucun CSV PixCell trouvé dans OUTPUTS_DIR/pixcell_out_histo/")


⚠️ Aucun CSV PixCell trouvé dans OUTPUTS_DIR/pixcell_out_histo/


# =========================================================
# 🧪 Tests supplémentaires pour couverture complète
# =========================================================


## Test 1: PRC activé/désactivé en mode pairé
Vérifier que PRC n'apparaît que quand `compute_prc=True` en mode pairé


In [17]:
# Test PRC activé (par défaut)
if 'tmp_csv' in locals() and Path(tmp_csv).exists():
    df_prc_on = run_eval_paired_experiment(
        name="test_prc_enabled",
        pairs_csv=tmp_csv,
        save_dir=METRICS_DIR,
        real_col=real_col if 'real_col' in locals() else "real_path",
        gen_col=gen_col if 'gen_col' in locals() else "synth_path",
        class_col=class_col if 'class_col' in locals() else "class",
        lpips_pairs=10,  # Petit échantillon pour test rapide
        compute_prc=True,  # PRC activé
        seed=SEED
    )

    print("✅ Colonnes avec PRC activé:", list(df_prc_on.columns))
    assert "Precision" in df_prc_on.columns, "Precision devrait être présente avec PRC activé"
    assert "Recall" in df_prc_on.columns, "Recall devrait être présente avec PRC activé"
    print("✅ PRC activé: OK")

    # Test PRC désactivé
    df_prc_off = run_eval_paired_experiment(
        name="test_prc_disabled",
        pairs_csv=tmp_csv,
        save_dir=METRICS_DIR,
        real_col=real_col if 'real_col' in locals() else "real_path",
        gen_col=gen_col if 'gen_col' in locals() else "synth_path",
        class_col=class_col if 'class_col' in locals() else "class",
        lpips_pairs=10,
        compute_prc=False,  # PRC désactivé
        seed=SEED
    )

    print("✅ Colonnes avec PRC désactivé:", list(df_prc_off.columns))
    assert "Precision" not in df_prc_off.columns, "Precision ne devrait PAS être présente avec PRC désactivé"
    assert "Recall" not in df_prc_off.columns, "Recall ne devrait PAS être présente avec PRC désactivé"
    print("✅ PRC désactivé: OK")
else:
    print("⚠️ tmp_csv non disponible, test PRC ignoré")


📑 CSV : /workspace/artifacts/metrics/pairs_real_synth.normalized.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : test_prc_enabled  |  Device: cuda
📦 Classes : ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']


Évaluation pairée: 100%|██████████| 9/9 [00:45<00:00,  5.01s/it]



✅ Rapport pairé enregistré : /workspace/artifacts/metrics/test_prc_enabled_paired_metrics.csv
✅ Colonnes avec PRC activé: ['class', 'n_pairs', 'FID', 'KID_mean', 'KID_std', 'LPIPS', 'FID_UNI', 'Precision', 'Recall']
✅ PRC activé: OK
📑 CSV : /workspace/artifacts/metrics/pairs_real_synth.normalized.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : test_prc_disabled  |  Device: cuda
📦 Classes : ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']


Évaluation pairée: 100%|██████████| 9/9 [00:45<00:00,  5.00s/it]


✅ Rapport pairé enregistré : /workspace/artifacts/metrics/test_prc_disabled_paired_metrics.csv
✅ Colonnes avec PRC désactivé: ['class', 'n_pairs', 'FID', 'KID_mean', 'KID_std', 'LPIPS', 'FID_UNI']
✅ PRC désactivé: OK


## Test 2: Vérification que PRC n'apparaît jamais en mode dossiers
Vérifier que les colonnes Precision/Recall ne sont jamais présentes en mode dossiers


In [18]:
# Test mode dossiers - vérifier absence de Precision/Recall
df_dirs = run_eval_experiment(
    name="test_mode_dossiers_no_prc",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=50,
    lpips_pairs=10,
    seed=SEED
)

print("✅ Colonnes en mode dossiers:", list(df_dirs.columns))
assert "Precision" not in df_dirs.columns, "Precision ne devrait JAMAIS être présente en mode dossiers"
assert "Recall" not in df_dirs.columns, "Recall ne devrait JAMAIS être présente en mode dossiers"
print("✅ Mode dossiers sans PRC: OK")


🎯 Expérience : test_mode_dossiers_no_prc
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:06<00:00,  6.17s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_mode_dossiers_no_prc_metrics.csv
✅ Colonnes en mode dossiers: ['class', 'n_real', 'n_synth', 'FID', 'KID_mean', 'KID_std', 'LPIPS', 'FID_UNI', 'experiment', 'max_images_per_class', 'lpips_pairs']
✅ Mode dossiers sans PRC: OK


## Test 3: Reproductibilité avec différentes seeds
Vérifier que les résultats sont reproductibles avec la même seed


In [19]:
# Test reproductibilité - même seed = mêmes résultats
df_seed1 = run_eval_experiment(
    name="test_repro_seed42_run1",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=50,
    lpips_pairs=10,
    seed=SEED
)

df_seed2 = run_eval_experiment(
    name="test_repro_seed42_run2",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=50,
    lpips_pairs=10,
    seed=SEED
)

# Vérifier que les résultats sont identiques
import numpy as np
for col in ["FID", "KID_mean", "LPIPS"]:
    if col in df_seed1.columns and col in df_seed2.columns:
        diff = np.abs(df_seed1[col].values - df_seed2[col].values)
        assert np.allclose(diff, 0, atol=1e-6), f"Colonne {col} diffère entre les deux runs avec seed=42"
        print(f"✅ {col}: reproductible (diff max: {diff.max():.2e})")

print("✅ Reproductibilité: OK")


🎯 Expérience : test_repro_seed42_run1
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:06<00:00,  6.28s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_repro_seed42_run1_metrics.csv
🎯 Expérience : test_repro_seed42_run2
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:06<00:00,  6.14s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_repro_seed42_run2_metrics.csv
✅ FID: reproductible (diff max: 0.00e+00)
✅ KID_mean: reproductible (diff max: 0.00e+00)
✅ LPIPS: reproductible (diff max: 0.00e+00)
✅ Reproductibilité: OK


## Test 4: FID_UNI activé/désactivé
Vérifier que FID_UNI n'apparaît que quand uni_embed est fourni


In [20]:
# Test avec FID_UNI activé
df_uni_on = run_eval_experiment(
    name="test_fid_uni_enabled",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=30,
    lpips_pairs=10,
    seed=SEED,
    uni_embed=uni_embed if 'uni_embed' in locals() else None,
    fid_uni_max=20
)

print("✅ Colonnes avec FID_UNI:", list(df_uni_on.columns))
if 'uni_embed' in locals():
    assert "FID_UNI" in df_uni_on.columns, "FID_UNI devrait être présente avec uni_embed"
    assert not df_uni_on["FID_UNI"].isna().all(), "FID_UNI ne devrait pas être tout NaN"
    print("✅ FID_UNI activé: OK")
else:
    print("⚠️ uni_embed non disponible, test FID_UNI ignoré")

# Test sans FID_UNI
df_uni_off = run_eval_experiment(
    name="test_fid_uni_disabled",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=30,
    lpips_pairs=10,
    seed=SEED,
    uni_embed=None,  # Pas de FID_UNI
)

print("✅ Colonnes sans FID_UNI:", list(df_uni_off.columns))
# FID_UNI peut être présente mais avec NaN
if "FID_UNI" in df_uni_off.columns:
    assert df_uni_off["FID_UNI"].isna().all(), "FID_UNI devrait être NaN sans uni_embed"
    print("✅ FID_UNI désactivé (NaN): OK")


🎯 Expérience : test_fid_uni_enabled
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:10<00:00, 10.77s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_fid_uni_enabled_metrics.csv
✅ Colonnes avec FID_UNI: ['class', 'n_real', 'n_synth', 'FID', 'KID_mean', 'KID_std', 'LPIPS', 'FID_UNI', 'experiment', 'max_images_per_class', 'lpips_pairs']
✅ FID_UNI activé: OK
🎯 Expérience : test_fid_uni_disabled
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:05<00:00,  5.96s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_fid_uni_disabled_metrics.csv
✅ Colonnes sans FID_UNI: ['class', 'n_real', 'n_synth', 'FID', 'KID_mean', 'KID_std', 'LPIPS', 'FID_UNI', 'experiment', 'max_images_per_class', 'lpips_pairs']
✅ FID_UNI désactivé (NaN): OK


## Test 5: Gestion des classes (sélection, intersection, drop_back_variant)


In [21]:
# Test avec classes spécifiques
df_classes_specific = run_eval_experiment(
    name="test_classes_specific",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM", "LYM"],  # Classes spécifiques
    max_images_per_class=30,
    lpips_pairs=10,
    seed=SEED
)

print("✅ Classes évaluées:", sorted(df_classes_specific["class"].unique()))
assert set(df_classes_specific["class"].unique()) == {"TUM", "LYM"}, "Seules TUM et LYM devraient être présentes"
print("✅ Classes spécifiques: OK")

# Test avec drop_back_variant
df_no_back = run_eval_experiment(
    name="test_drop_back",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=None,  # Toutes les classes
    drop_back_variant=True,  # Exclure BACK
    max_images_per_class=30,
    lpips_pairs=10,
    seed=SEED
)

print("✅ Classes sans BACK:", sorted(df_no_back["class"].unique()))
assert "BACK" not in df_no_back["class"].values, "BACK ne devrait pas être présent avec drop_back_variant=True"
print("✅ drop_back_variant: OK")


🎯 Expérience : test_classes_specific
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM', 'LYM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM, LYM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM_LYM.csv


Évaluation multi-classes: 100%|██████████| 2/2 [00:11<00:00,  5.97s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM_LYM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_classes_specific_metrics.csv
✅ Classes évaluées: ['LYM', 'TUM']
✅ Classes spécifiques: OK
🎯 Expérience : test_drop_back
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['ADI', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : ADI, DEB, LYM, MUC, MUS, NORM, STR, TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv


Évaluation multi-classes: 100%|██████████| 8/8 [00:45<00:00,  5.73s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_ADI_DEB_LYM_MUC_MUS_NORM_STR_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_drop_back_metrics.csv
✅ Classes sans BACK: ['ADI', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
✅ drop_back_variant: OK


## Test 6: Mode pairé avec CSV sans colonne de classe (regroupement en 'ALL')


In [22]:
# Créer un CSV temporaire sans colonne de classe
if 'tmp_csv' in locals():
    df_test = pd.read_csv(tmp_csv)
    # Supprimer temporairement la colonne de classe
    if class_col in df_test.columns:
        df_test_no_class = df_test.drop(columns=[class_col])
        tmp_csv_no_class = METRICS_DIR / "pairs_test_no_class.csv"
        df_test_no_class.to_csv(tmp_csv_no_class, index=False)
        
        df_all = run_eval_paired_experiment(
            name="test_no_class_col",
            pairs_csv=tmp_csv_no_class,
            save_dir=METRICS_DIR,
            real_col=real_col,
            gen_col=gen_col,
            class_col=None,  # Pas de colonne de classe
            lpips_pairs=10,
            seed=SEED
        )
        
        print("✅ Classes détectées:", sorted(df_all["class"].unique()))
        assert "ALL" in df_all["class"].values or len(df_all["class"].unique()) == 1, "Devrait regrouper en 'ALL'"
        print("✅ Regroupement en 'ALL': OK")
    else:
        print("⚠️ Colonne de classe non trouvée dans le CSV de test")
else:
    print("⚠️ tmp_csv non disponible, test ignoré")


📑 CSV : /workspace/artifacts/metrics/pairs_test_no_class.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : test_no_class_col  |  Device: cuda
📦 Classes : ['ALL']


Évaluation pairée: 100%|██████████| 1/1 [00:05<00:00,  5.22s/it]


✅ Rapport pairé enregistré : /workspace/artifacts/metrics/test_no_class_col_paired_metrics.csv
✅ Classes détectées: ['ALL']
✅ Regroupement en 'ALL': OK


## Test 7: Validation des erreurs (chemins invalides, classes manquantes)


In [23]:
# Test avec chemin invalide
try:
    evaluator = FIDLPIPSEvaluator(
        real_root="/chemin/inexistant",
        gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
        classes=["TUM"]
    )
    assert False, "Devrait lever FileNotFoundError"
except FileNotFoundError as e:
    print(f"✅ Erreur chemin invalide capturée: {type(e).__name__}")

# Test avec classe inexistante
try:
    evaluator = FIDLPIPSEvaluator(
        real_root=DATA_ROOT / "NCT-CRC-HE-100K",
        gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
        classes=["CLASSE_INEXISTANTE"]
    )
    assert False, "Devrait lever ValueError"
except ValueError as e:
    print(f"✅ Erreur classe inexistante capturée: {type(e).__name__}")

# Test avec gen_root None
try:
    evaluator = FIDLPIPSEvaluator(
        real_root=DATA_ROOT / "NCT-CRC-HE-100K",
        gen_root=None,
        classes=["TUM"]
    )
    assert False, "Devrait lever FileNotFoundError"
except FileNotFoundError as e:
    print(f"✅ Erreur gen_root None capturée: {type(e).__name__}")

print("✅ Validation des erreurs: OK")


✅ Erreur chemin invalide capturée: FileNotFoundError
✅ Erreur classe inexistante capturée: ValueError
✅ Erreur gen_root None capturée: FileNotFoundError
✅ Validation des erreurs: OK


## Test 8: Différentes valeurs de lpips_pairs et max_images_per_class


In [24]:
# Test avec lpips_pairs limité
df_lpips_limited = run_eval_experiment(
    name="test_lpips_limited",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=100,
    lpips_pairs=5,  # Très limité
    seed=SEED
)

print(f"✅ LPIPS avec lpips_pairs=5: {df_lpips_limited['LPIPS'].values[0]:.6f}")
assert not np.isnan(df_lpips_limited['LPIPS'].values[0]), "LPIPS ne devrait pas être NaN"
print("✅ lpips_pairs limité: OK")

# Test avec max_images_per_class petit
df_small_max = run_eval_experiment(
    name="test_small_max_images",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=10,  # Très petit
    lpips_pairs=5,
    seed=SEED
)

print(f"✅ n_real avec max_images=10: {df_small_max['n_real'].values[0]}")
assert df_small_max['n_real'].values[0] <= 10, "n_real ne devrait pas dépasser max_images_per_class"
print("✅ max_images_per_class: OK")


🎯 Expérience : test_lpips_limited
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:07<00:00,  7.23s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_lpips_limited_metrics.csv
✅ LPIPS avec lpips_pairs=5: 0.000000
✅ lpips_pairs limité: OK
🎯 Expérience : test_small_max_images
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:05<00:00,  5.52s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_small_max_images_metrics.csv
✅ n_real avec max_images=10: 10
✅ max_images_per_class: OK


## Test 9: Cohérence des colonnes entre modes
Vérifier que les colonnes sont cohérentes entre mode dossiers et mode pairé


In [25]:
# Colonnes attendues en mode dossiers
expected_cols_dirs = {"class", "n_real", "n_synth", "FID", "KID_mean", "KID_std", "LPIPS", "FID_UNI"}
df_test_dirs = run_eval_experiment(
    name="test_cols_dirs",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=30,
    lpips_pairs=10,
    seed=SEED
)

actual_cols_dirs = set(df_test_dirs.columns) - {"experiment", "max_images_per_class", "lpips_pairs"}
print(f"✅ Colonnes mode dossiers: {sorted(actual_cols_dirs)}")
assert expected_cols_dirs.issubset(actual_cols_dirs), f"Colonnes manquantes: {expected_cols_dirs - actual_cols_dirs}"
assert "Precision" not in actual_cols_dirs, "Precision ne devrait pas être en mode dossiers"
assert "Recall" not in actual_cols_dirs, "Recall ne devrait pas être en mode dossiers"
print("✅ Colonnes mode dossiers: OK")

# Colonnes attendues en mode pairé (avec PRC)
if 'tmp_csv' in locals():
    df_test_paired = run_eval_paired_experiment(
        name="test_cols_paired",
        pairs_csv=tmp_csv,
        save_dir=METRICS_DIR,
        real_col=real_col,
        gen_col=gen_col,
        class_col=class_col,
        lpips_pairs=10,
        compute_prc=True,
        seed=SEED
    )
    
    expected_cols_paired = {"class", "n_pairs", "FID", "KID_mean", "KID_std", "LPIPS", "FID_UNI", "Precision", "Recall"}
    actual_cols_paired = set(df_test_paired.columns)
    print(f"✅ Colonnes mode pairé (avec PRC): {sorted(actual_cols_paired)}")
    assert expected_cols_paired.issubset(actual_cols_paired), f"Colonnes manquantes: {expected_cols_paired - actual_cols_paired}"
    print("✅ Colonnes mode pairé: OK")
else:
    print("⚠️ tmp_csv non disponible, test colonnes pairé ignoré")


🎯 Expérience : test_cols_dirs
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:05<00:00,  5.89s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_cols_dirs_metrics.csv
✅ Colonnes mode dossiers: ['FID', 'FID_UNI', 'KID_mean', 'KID_std', 'LPIPS', 'class', 'n_real', 'n_synth']
✅ Colonnes mode dossiers: OK
📑 CSV : /workspace/artifacts/metrics/pairs_real_synth.normalized.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : test_cols_paired  |  Device: cuda
📦 Classes : ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']


Évaluation pairée: 100%|██████████| 9/9 [00:45<00:00,  5.03s/it]


✅ Rapport pairé enregistré : /workspace/artifacts/metrics/test_cols_paired_paired_metrics.csv
✅ Colonnes mode pairé (avec PRC): ['FID', 'FID_UNI', 'KID_mean', 'KID_std', 'LPIPS', 'Precision', 'Recall', 'class', 'n_pairs']
✅ Colonnes mode pairé: OK


## Test 10: Sanity check - valeurs attendues
Vérifier que les métriques ont des valeurs raisonnables


In [26]:
# Test réel vs réel - FID et LPIPS devraient être proches de 0
df_sanity = run_eval_experiment(
    name="test_sanity_check",
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM"],
    max_images_per_class=50,
    lpips_pairs=20,
    seed=SEED
)

fid_val = df_sanity["FID"].values[0]
lpips_val = df_sanity["LPIPS"].values[0]

print(f"✅ FID (réel vs réel): {fid_val:.6f}")
print(f"✅ LPIPS (réel vs réel): {lpips_val:.6f}")

# FID devrait être très proche de 0 (même distribution)
assert abs(fid_val) < 1.0, f"FID réel vs réel devrait être < 1.0, obtenu: {fid_val}"
# LPIPS devrait être très proche de 0 (mêmes images)
assert lpips_val < 0.1, f"LPIPS réel vs réel devrait être < 0.1, obtenu: {lpips_val}"

print("✅ Sanity check valeurs: OK")


🎯 Expérience : test_sanity_check
📁 real_root = /workspace/data/NCT-CRC-HE-100K
📁 gen_root  = /workspace/data/NCT-CRC-HE-100K
📦 classes évaluées : ['TUM']
🎲 seed = 42
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:06<00:00,  6.22s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
💾 Résultats sauvegardés → /workspace/artifacts/metrics/test_sanity_check_metrics.csv
✅ FID (réel vs réel): -0.000059
✅ LPIPS (réel vs réel): 0.000000
✅ Sanity check valeurs: OK


## Test 11: Mode pairé avec lpips_pairs=None (toutes les paires)


In [27]:
# Test avec toutes les paires (lpips_pairs=None)
if 'tmp_csv' in locals():
    df_all_pairs = run_eval_paired_experiment(
        name="test_all_pairs",
        pairs_csv=tmp_csv,
        save_dir=METRICS_DIR,
        real_col=real_col,
        gen_col=gen_col,
        class_col=class_col,
        lpips_pairs=None,  # Toutes les paires
        seed=SEED
    )
    
    # Vérifier que toutes les paires disponibles sont utilisées
    df_source = pd.read_csv(tmp_csv)
    if class_col and class_col in df_source.columns:
        for cls in df_all_pairs["class"].unique():
            n_expected = len(df_source[df_source[class_col] == cls])
            n_actual = df_all_pairs[df_all_pairs["class"] == cls]["n_pairs"].values[0]
            print(f"✅ Classe {cls}: {n_actual} paires utilisées (sur {n_expected} disponibles)")
            # Peut être inférieur si certaines images sont invalides
            assert n_actual <= n_expected, f"n_pairs ({n_actual}) ne devrait pas dépasser le nombre de lignes ({n_expected})"
    
    print("✅ lpips_pairs=None (toutes les paires): OK")
else:
    print("⚠️ tmp_csv non disponible, test lpips_pairs=None ignoré")


📑 CSV : /workspace/artifacts/metrics/pairs_real_synth.normalized.csv  (5625 lignes)
🔍 Colonnes détectées : ['global_index', 'split', 'ref_path', 'ref_label', 'ref_label_full', 'ref_color', 'seed', 'out_path', 'guidance_scale', 'steps']
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
🎯 Expérience pairée : test_all_pairs  |  Device: cuda
📦 Classes : ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']


Évaluation pairée: 100%|██████████| 9/9 [03:43<00:00, 24.87s/it]


✅ Rapport pairé enregistré : /workspace/artifacts/metrics/test_all_pairs_paired_metrics.csv
✅ Classe ADI: 625 paires utilisées (sur 625 disponibles)
✅ Classe BACK: 625 paires utilisées (sur 625 disponibles)
✅ Classe DEB: 625 paires utilisées (sur 625 disponibles)
✅ Classe LYM: 625 paires utilisées (sur 625 disponibles)
✅ Classe MUC: 625 paires utilisées (sur 625 disponibles)
✅ Classe MUS: 625 paires utilisées (sur 625 disponibles)
✅ Classe NORM: 625 paires utilisées (sur 625 disponibles)
✅ Classe STR: 625 paires utilisées (sur 625 disponibles)
✅ Classe TUM: 625 paires utilisées (sur 625 disponibles)
✅ lpips_pairs=None (toutes les paires): OK


## Test 12: FIDLPIPSEvaluator direct (mono-classe et multi-classes)


In [28]:
# Test mono-classe
evaluator_mono = FIDLPIPSEvaluator(
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes="TUM",  # String au lieu de liste
    max_images=30,
    lpips_pairs=10,
    seed=SEED
)
df_mono = evaluator_mono.run()
print(f"✅ Mono-classe (string): {len(df_mono)} classe(s)")
assert len(df_mono) == 1, "Devrait avoir exactement 1 classe"
assert df_mono["class"].values[0] == "TUM", "Classe devrait être TUM"
print("✅ Mono-classe: OK")

# Test multi-classes
evaluator_multi = FIDLPIPSEvaluator(
    real_root=DATA_ROOT / "NCT-CRC-HE-100K",
    gen_root=DATA_ROOT / "NCT-CRC-HE-100K",
    save_dir=METRICS_DIR,
    classes=["TUM", "LYM", "MUC"],  # Liste
    max_images=30,
    lpips_pairs=10,
    seed=SEED
)
df_multi = evaluator_multi.run()
print(f"✅ Multi-classes: {len(df_multi)} classe(s)")
assert len(df_multi) == 3, "Devrait avoir exactement 3 classes"
assert set(df_multi["class"].values) == {"TUM", "LYM", "MUC"}, "Classes devraient être TUM, LYM, MUC"
print("✅ Multi-classes: OK")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv


Évaluation multi-classes: 100%|██████████| 1/1 [00:05<00:00,  5.73s/it]



✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM.csv
✅ Mono-classe (string): 1 classe(s)
✅ Mono-classe: OK
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth
✅ Initialisé : NCT-CRC-HE-100K vs NCT-CRC-HE-100K
📂 Classes évaluées : TUM, LYM, MUC
💾 Rapport : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM_LYM_MUC.csv


Évaluation multi-classes: 100%|██████████| 3/3 [00:17<00:00,  5.68s/it]


✅ Rapport enregistré : /workspace/artifacts/metrics/NCT-CRC-HE-100K_NCT-CRC-HE-100K_TUM_LYM_MUC.csv
✅ Multi-classes: 3 classe(s)
✅ Multi-classes: OK


## Résumé des tests


In [29]:
print("""
✅ Tests de couverture complète :

1. ✅ PRC activé/désactivé en mode pairé
2. ✅ Vérification absence PRC en mode dossiers
3. ✅ Reproductibilité avec seeds
4. ✅ FID_UNI activé/désactivé
5. ✅ Gestion des classes (sélection, drop_back_variant)
6. ✅ Mode pairé sans colonne de classe (regroupement 'ALL')
7. ✅ Validation des erreurs (chemins invalides, classes manquantes)
8. ✅ Différentes valeurs de lpips_pairs et max_images_per_class
9. ✅ Cohérence des colonnes entre modes
10. ✅ Sanity check - valeurs attendues (réel vs réel)
11. ✅ Mode pairé avec lpips_pairs=None
12. ✅ FIDLPIPSEvaluator direct (mono et multi-classes)

Tous les tests passent ! 🎉
""")



✅ Tests de couverture complète :

1. ✅ PRC activé/désactivé en mode pairé
2. ✅ Vérification absence PRC en mode dossiers
3. ✅ Reproductibilité avec seeds
4. ✅ FID_UNI activé/désactivé
5. ✅ Gestion des classes (sélection, drop_back_variant)
6. ✅ Mode pairé sans colonne de classe (regroupement 'ALL')
7. ✅ Validation des erreurs (chemins invalides, classes manquantes)
8. ✅ Différentes valeurs de lpips_pairs et max_images_per_class
9. ✅ Cohérence des colonnes entre modes
10. ✅ Sanity check - valeurs attendues (réel vs réel)
11. ✅ Mode pairé avec lpips_pairs=None
12. ✅ FIDLPIPSEvaluator direct (mono et multi-classes)

Tous les tests passent ! 🎉

